In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import statsforecast
from statsforecast.models import SeasonalNaive

In [ ]:
class RedemptionModel_2:

    def __init__(self, X, target_col):
        '''
        Args:
        X (pandas.DataFrame): Dataset of predictors, output from load_data()
        target_col (str): column name for target variable
        '''
        self._predictions = {}
        self.X = X
        self.target_col = target_col
        self.results = {} # dict of dicts with model results

    def score(self, truth, preds):
        # Score our predictions - modify this method as you like
        return MAPE(truth, preds)


    def run_models(self, n_splits=4, test_size=365):
        '''Run the models and store results for cross validated splits in
        self.results.
        '''
        # Time series split
        tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size)
        cnt = 0 # keep track of splits
        for train, test in tscv.split(self.X):
            X_train = self.X.iloc[train]
            X_test = self.X.iloc[test]
            # Base model - please leave this here
            preds = self._base_model(X_train, X_test)
            if 'Base' not in self.results:
                self.results['Base'] = {}
            self.results['Base'][cnt] = self.score(X_test[self.target_col],
                                preds)
            self.plot(preds, 'Base', 'red')

            # Other models...
            # Seasonal Naive
            preds_sn = self._seasonal_naive(X_train, X_test)
            if 'SeasonalNaive' not in self.results:
                self.results['SeasonalNaive'] = {}
            self.results['SeasonalNaive'][cnt] = self.score(X_test[self.target_col], preds_sn)
            self.plot(preds_sn, 'SeasonalNaive', 'lightsalmon')

            # Holt Winters
            preds_hw = self._Holt_Winters(X_train, X_test)
            if 'HoltWinters' not in self.results:
                self.results['HoltWinters'] = {}
            self.results['HoltWinters'][cnt] = self.score(X_test[self.target_col], preds_hw)
            self.plot(preds_hw, 'HoltWinters', 'skyblue')
            cnt += 1


    def _base_model(self, train, test):
        '''
        Our base, too-simple model.
        Your model needs to take the training and test datasets (dataframes)
        and output a prediction based on the test data.

        Please leave this method as-is.

        '''
        res = sm.tsa.seasonal_decompose(train[self.target_col],
                                        period=365)
        res_clip = res.seasonal.apply(lambda x: max(0,x))
        res_clip.index = res_clip.index.dayofyear
        res_clip = res_clip.groupby(res_clip.index).mean()
        res_dict = res_clip.to_dict()
        return pd.Series(index = test.index,
                         data = map(lambda x: res_dict[x], test.index.dayofyear))

    def _seasonal_naive(self, train, test):
        SN = statsforecast.models.SeasonalNaive(season_length=365).fit(train[self.target_col])
        y_SN = SN.predict(h=test.shape[0]).get('mean')
        return pd.Series(index = test.index, data = y_SN)

    def _Holt_Winters(self, train, test):
        HW = statsforecast.models.HoltWinters(error_type='A', season_length = 365).fit(train[self.target_col])
        y_HW = HW.predict(h=test.shape[0]).get('mean')
        return pd.Series(index = test.index, data = y_HW)

    def plot(self, preds, label, color):
        # plot out the forecasts
        fig, ax = plt.subplots(figsize=(15, 5))
        ax.scatter(self.X.index, self.X[self.target_col], s=0.4, color='grey',
            label='Observed')
        ax.plot(preds, label = label, color=color)
        plt.legend()